In [1]:

!pip install vaderSentiment
!pip install nltk
!pip install gensim


# VADER (Valence Aware Dictionary and sEntiment Reasoner) 

is a lexicon and rule-based sentiment analysis tool specifically attuned to sentiments expressed in social media. It is designed to perform well on short texts, which makes it ideal for analyzing comments, tweets, and other short-form content.

In [2]:

import sys
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from gensim import corpora, models
from sklearn.feature_extraction.text import CountVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Ensure necessary nltk resources are downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Add the directory containing config.py to the Python path
sys.path.append(os.path.abspath(os.path.join('..', '..')))
from config import REDDIT_RAW_DATA_DIR, GITHUB_RAW_DATA_DIR, STACKOVERFLOW_RAW_DATA_DIR, IMAGE_DIR, INTERMEDIATE_ANALYSIS_DIR

# Define data directories
DATA_DIRS = {
    'Reddit': REDDIT_RAW_DATA_DIR,
    'GitHub': GITHUB_RAW_DATA_DIR,
    'StackOverflow': STACKOVERFLOW_RAW_DATA_DIR
}

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()


[nltk_data] Downloading package stopwords to /Users/tnathu-
[nltk_data]     ai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tnathu-ai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
%%time

# Function to preprocess text
def preprocess(text):
    """Preprocess the text by converting to lowercase, removing punctuation, and eliminating stopwords."""
    if not isinstance(text, str):
        return ""
    stop_words = set(stopwords.words('english'))
    text = ''.join([c.lower() for c in text if c.isalnum() or c.isspace()])
    return ' '.join([word for word in text.split() if word not in stop_words])

# Function to generate n-grams from text
def get_ngrams(text, n=(2, 3)):
    """Generate n-grams from the text."""
    if not text.strip():
        return ""
    vectorizer = CountVectorizer(ngram_range=n)
    try:
        ngrams = vectorizer.fit_transform([text])
        ngram_features = vectorizer.get_feature_names_out()
        return ' '.join(ngram_features)
    except ValueError:
        return ""

# Function to get sentiment polarity using VADER
def get_sentiment(text):
    """Return the compound sentiment score of the text using VADER."""
    if text:
        sentiment_score = analyzer.polarity_scores(text)
        return sentiment_score['compound']
    return 0

# Function to perform sentiment analysis on the text column of a DataFrame
def perform_sentiment_analysis(df, text_column):
    """Perform sentiment analysis on the specified text column."""
    df['Processed Text'] = df[text_column].apply(preprocess)
    df['Ngrams'] = df['Processed Text'].apply(lambda x: get_ngrams(x, (2, 3)))
    df['Sentiment'] = df['Ngrams'].apply(get_sentiment)
    return df

# Function to perform topic modeling using LDA
def topic_modeling(df, n=(2, 3)):
    """Perform LDA topic modeling on the processed comments using n-grams."""
    if df.empty or 'Ngrams' not in df.columns:
        return []
    texts = [text.split() for text in df['Ngrams'] if text.strip()]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=50, iterations=100)
    return lda_model.print_topics(num_words=5)

# Function to perform qualitative analysis to understand specific reasons behind the sentiments
def qualitative_analysis(df, text_column, output_file):
    """Perform qualitative analysis to understand specific reasons behind the sentiments."""
    with open(output_file, 'w') as f:
        for index, row in df.iterrows():
            f.write(f"Title: {row.get(text_column)}\n")
            f.write(f"Sentiment: {row.get('Sentiment')}\n")
            f.write(f"Comment: {row.get('Processed Text')}\n")
            f.write("\n" + "-"*40 + "\n")

# Function to perform time series analysis
def time_series_analysis(df, date_column):
    """Perform time series analysis on the specified date column."""
    df[date_column] = pd.to_datetime(df[date_column])
    time_series_df = df.groupby(df[date_column].dt.date).mean()
    time_series_df = time_series_df.reset_index()
    return time_series_df

# Function to visualize time series data using Seaborn
def visualize_time_series(time_series_df, date_column, value_column, source_type, topic):
    """Visualize the time series data using Seaborn and save the plot."""
    sns.set(style="darkgrid")
    plt.figure(figsize=(20, 10))
    sns.lineplot(x=date_column, y=value_column, data=time_series_df)
    plt.title(f"Time Series Analysis - {source_type} - Topic: {topic}")
    plt.xlabel("Date")
    plt.ylabel(value_column.capitalize())

    output_dir = IMAGE_DIR
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, f"{source_type.lower()}_time_series_analysis_{topic}.png")
    plt.savefig(output_file)
    plt.close()
    print(f"Saved time series visualization for {source_type} - Topic: {topic} to {output_file}")

# Function to get top services based on the specified metric
def get_top_services(df, service_column, metric_column, top_n=10):
    """Get the top services based on the specified metric."""
    top_services = df.groupby(service_column)[metric_column].sum().nlargest(top_n).reset_index()
    return top_services

# Function to process and visualize top services
def process_and_visualize_top_services(directory, source_type, service_column, metric_column, top_n=10):
    """Process each CSV file to visualize top services."""
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            
            if service_column in df.columns and metric_column in df.columns:
                top_services_df = get_top_services(df, service_column, metric_column, top_n)
                sns.set(style="whitegrid")
                plt.figure(figsize=(15, 8))
                sns.barplot(x=metric_column, y=service_column, data=top_services_df, palette="viridis")
                plt.title(f"Top {top_n} Services by {metric_column.capitalize()} - {source_type}")
                plt.xlabel(metric_column.capitalize())
                plt.ylabel(service_column.capitalize())
                
                output_dir = IMAGE_DIR
                os.makedirs(output_dir, exist_ok=True)
                output_file = os.path.join(output_dir, f"{source_type.lower()}_top_services.png")
                plt.savefig(output_file)
                plt.close()
                print(f"Saved top services visualization for {source_type} to {output_file}")
            else:
                print(f"No suitable service column or metric column found in {file_path}")

# Function to process each CSV file in the specified directory and perform the analyses
def process_files(directory, source_type):
    """Process each CSV file in the specified directory based on source type."""
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)

            text_column = None
            title_column = None
            date_column = None

            if source_type == 'Reddit':
                text_column = 'Comment'
                title_column = 'Title'
                date_column = 'Comment Date'
            elif source_type == 'GitHub':
                text_column = 'Description'
                title_column = 'Repository Name'
                date_column = 'Last Updated'
            elif source_type == 'StackOverflow':
                text_column = 'Question Body'
                title_column = 'Question Title'
                date_column = 'Question Creation Date'
            
            if text_column in df.columns and date_column in df.columns:
                # Perform sentiment analysis
                df = perform_sentiment_analysis(df, text_column)
                analysis_file = os.path.join(INTERMEDIATE_ANALYSIS_DIR, f"{source_type.lower()}_sentiment_analysis.txt")
                with open(analysis_file, 'w') as f:
                    f.write(df.to_string())
                print(f"Saved sentiment analysis for {source_type} to {analysis_file}")

                # Perform topic modeling
                topics = topic_modeling(df)
                topic_file = os.path.join(INTERMEDIATE_ANALYSIS_DIR, f"{source_type.lower()}_topic_modeling.txt")
                with open(topic_file, 'w') as f:
                    for topic in topics:
                        f.write(f"{topic}\n")
                        print(f"Extracted Topic: {topic}")
                print(f"Saved topic modeling for {source_type} to {topic_file}")

                # Perform qualitative analysis
                qualitative_file = os.path.join(INTERMEDIATE_ANALYSIS_DIR, f"{source_type.lower()}_qualitative_analysis.txt")
                qualitative_analysis(df, text_column, qualitative_file)
                print(f"Saved qualitative analysis for {source_type} to {qualitative_file}")

                # Perform time series analysis for each topic
                for topic in topics:
                    topic_words = ' '.join(word.split('*')[1].replace('"', '') for word in topic[1].split('+')[:3])
                    topic_df = df[df['Ngrams'].str.contains('|'.join(topic_words.split()))]
                    if not topic_df.empty:
                        time_series_df = time_series_analysis(topic_df, date_column)
                        visualize_time_series(time_series_df, date_column, 'Sentiment', source_type, topic_words)
            else:
                print(f"No suitable text column or date column found in {file_path}")

# Function to execute the script for each platform separately to handle kernel breaks and perform granular analysis
def execute_for_platform(platform):
    """Execute the analysis for a specific platform."""
    directory = DATA_DIRS.get(platform)
    if directory:
        process_files(directory, platform)
        if platform == 'Reddit':
            process_and_visualize_top_services(directory, platform, 'Title', 'Comment Upvotes')
        elif platform == 'GitHub':
            process_and_visualize_top_services(directory, platform, 'Repository Name', 'Stars')
        elif platform == 'StackOverflow':
            process_and_visualize_top_services(directory, platform, 'Question Title', 'Question Upvote')

# Execute the script for each platform separately
if __name__ == "__main__":
    platforms = [ 'GitHub', 'StackOverflow', 'Reddit']
    for platform in platforms:
        execute_for_platform(platform)


Saved sentiment analysis for GitHub to /Users/tnathu-ai/VSCode/Government-Digital-Transformation/data/intermediate/analysis/github_sentiment_analysis.txt
Extracted Topic: (0, '0.060*"data" + 0.032*"based" + 0.032*"study" + 0.032*"using" + 0.032*"road"')
Extracted Topic: (1, '0.025*"government" + 0.025*"us" + 0.025*"location" + 0.025*"information" + 0.025*"forecast"')
Extracted Topic: (2, '0.047*"market" + 0.022*"taxi" + 0.021*"booking" + 0.020*"type" + 0.013*"uber"')
Extracted Topic: (3, '0.003*"market" + 0.003*"data" + 0.003*"taxi" + 0.003*"based" + 0.003*"trends"')
Extracted Topic: (4, '0.003*"market" + 0.003*"booking" + 0.003*"type" + 0.003*"taxi" + 0.003*"information"')
Saved topic modeling for GitHub to /Users/tnathu-ai/VSCode/Government-Digital-Transformation/data/intermediate/analysis/github_topic_modeling.txt
Saved qualitative analysis for GitHub to /Users/tnathu-ai/VSCode/Government-Digital-Transformation/data/intermediate/analysis/github_qualitative_analysis.txt
Saved time se

<timed exec>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Saved time series visualization for GitHub - Topic: market  taxi  booking  to /Users/tnathu-ai/VSCode/Government-Digital-Transformation/assets/images/github_time_series_analysis_market  taxi  booking .png
Saved time series visualization for GitHub - Topic: market  data  taxi  to /Users/tnathu-ai/VSCode/Government-Digital-Transformation/assets/images/github_time_series_analysis_market  data  taxi .png


<timed exec>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Saved time series visualization for GitHub - Topic: market  booking  type  to /Users/tnathu-ai/VSCode/Government-Digital-Transformation/assets/images/github_time_series_analysis_market  booking  type .png
Saved sentiment analysis for GitHub to /Users/tnathu-ai/VSCode/Government-Digital-Transformation/data/intermediate/analysis/github_sentiment_analysis.txt
Extracted Topic: (0, '0.004*"healthcare" + 0.004*"care" + 0.004*"devices" + 0.004*"health" + 0.004*"patient"')
Extracted Topic: (1, '0.037*"healthcare" + 0.026*"care" + 0.020*"health" + 0.020*"devices" + 0.017*"patient"')
Extracted Topic: (2, '0.004*"healthcare" + 0.004*"care" + 0.004*"devices" + 0.004*"patient" + 0.004*"health"')
Extracted Topic: (3, '0.004*"healthcare" + 0.004*"care" + 0.004*"health" + 0.004*"devices" + 0.004*"tracking"')
Extracted Topic: (4, '0.004*"healthcare" + 0.004*"devices" + 0.004*"health" + 0.004*"care" + 0.004*"patient"')
Saved topic modeling for GitHub to /Users/tnathu-ai/VSCode/Government-Digital-Transfo

...

Saved time series visualization for Reddit - Topic: people  dont  like  to /Users/tnathu-ai/VSCode/Government-Digital-Transformation/assets/images/reddit_time_series_analysis_people  dont  like .png


<timed exec>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Saved time series visualization for Reddit - Topic: law  would  use  to /Users/tnathu-ai/VSCode/Government-Digital-Transformation/assets/images/reddit_time_series_analysis_law  would  use .png


<timed exec>:123: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.


: 